In [ ]:
import pandas as pd
import os
import piexif
from fractions import Fraction

def geotag_images(csv_filename, image_folder, image_extension='.JPG'):
    # Get the current working directory (where the script is located)
    script_directory = os.getcwd()

    # Construct the full path to the CSV file and image folder
    csv_file = os.path.join(script_directory, csv_filename)
    image_folder = os.path.join(script_directory, image_folder)

    # Read CSV file into a Pandas DataFrame
    df = pd.read_csv(csv_file)

    # Get a list of filenames in the image folder
    image_files = [f for f in os.listdir(image_folder)]

    # Iterate through rows in the DataFrame
    for index, row in df.iterrows():
        # Extract information from the current row
        name = row['Name']
        latitude = row['Latitude']
        longitude = row['Longitude']
        altitude = row['Altitude']

        # Check if a matching file is found (case-insensitive substring check)
        matching_files = [f for f in image_files if name.lower() in f.lower()]

        if not matching_files:
            print(f"Image file not found for {name}")
            continue

        # Use the first matching file (assuming there's only one match)
        matched_filename = matching_files[0]

        # Construct the full path to the corresponding image
        image_path = os.path.join(image_folder, matched_filename)

        # Read existing EXIF data
        exif_dict = piexif.load(image_path)

        # Convert latitude, longitude, and altitude to EXIF format
        lat_ref = "N" if latitude >= 0 else "S"
        lon_ref = "E" if longitude >= 0 else "W"

        gps_ifd = {
            piexif.GPSIFD.GPSLatitude: exif_gps_coord(latitude),
            piexif.GPSIFD.GPSLongitude: exif_gps_coord(longitude),
            piexif.GPSIFD.GPSAltitude: (int(altitude * 1000), 1000),
            piexif.GPSIFD.GPSAltitudeRef: 0,  # 0 indicates above sea level
            piexif.GPSIFD.GPSLatitudeRef: lat_ref,
            piexif.GPSIFD.GPSLongitudeRef: lon_ref,
        }

        # Add or update GPS data in the EXIF dictionary
        exif_dict["GPS"] = gps_ifd

        # Save the modified EXIF data to the image
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, image_path)

        print(f"Geotagged {matched_filename} with Latitude: {latitude}, Longitude: {longitude}, Altitude: {altitude}")

    # Print out the list of files in the image folder
    print(f"Files in the folder: {image_files}")

def exif_gps_coord(coord):
    degrees = abs(int(coord))
    minutes = int((abs(coord) - degrees) * 60)
    seconds = int(((abs(coord) - degrees) * 60 - minutes) * 60)

    return (
        (degrees, 1),
        (minutes, 1),
        (seconds, 1)
    )

# Example usage with your specific file names
geotag_images('filename.csv', 'Photosfoldername')